In [31]:
import pathlib

import rasterio as rio

from rasterio import Affine
import rasterio.warp
import geopandas as gpd
import numpy as np

In [25]:
data_dir = pathlib.Path('~/data/vcl/bodem-202311/vanNienke').expanduser()

# bathymetry
in_file = 'huidige_bodem_voor_maquette.tif'


# in_file = 'https://service.pdok.nl/hwh/luchtfotorgb/wmts/v1_0'
out_file = 'test-tile.tif'

In [26]:
dst_width = 1920
dst_height = 1200

In [27]:
extent_df = gpd.read_file(data_dir / 'bounding_box.shp')
extent_geom = extent_df.iloc[0]['geometry']
dst_left, dst_lower, dst_right, dst_upper = extent_geom.bounds

In [28]:
with rio.open(data_dir / in_file) as src:
    # Get the old transform and crs
    src_transform = src.transform 
    src_bounds = src.bounds
    src_width = src.width
    src_height = src.height
    
src_bounds
src_width_m = src_bounds.right - src_bounds.left
src_width_m
dst_width_m = dst_right - dst_left

src_width, src_width_m, dst_width_m

(4037, 80740.0, 33036.620488757675)

In [34]:

angle = -15
#adj_width = 0
# adj_height = 0

shift_x = dst_width / 2
shift_y =  - dst_width / 2
scale_x = 0.8
scale_y = 0.8

# these parameters should be computable using an example like:
# https://rasterio.readthedocs.io/en/latest/topics/virtual-warping.html

# see also:
# https://rasterio.readthedocs.io/en/stable/topics/reproject.html

with rio.open(data_dir / in_file) as src:
    # Get the old transform and crs
    src_transform = src.transform 
    crs = src.crs
    
    # Affine transformations for rotation and translation
    rotate = Affine.rotation(angle)
    trans_x = Affine.translation(shift_x,0)
    trans_y = Affine.translation(0, -shift_y)
    scale = Affine.scale(scale_x, scale_y)
    
    
    # Combine affine transformations
    dst_transform = src_transform * scale * rotate * trans_x * trans_y 
    
    # Get band data
    band = np.array(src.read(1))
    
    # Get the new shape
    y, x = band.shape
    # dst_height = y + adj_height
    # dst_width = x + adj_width
    
    # set properties for output
    dst_kwargs = src.meta.copy()
    dst_kwargs.update(
        {
            "transform": dst_transform,
            "height": dst_height,
            "width": dst_width,
            "nodata": 0,  
        }
    )
    
    # write to disk
    with rio.open(out_file, "w", **dst_kwargs) as dst:
        # reproject to new CRS
    
        rasterio.warp.reproject(
            source=band,
            destination=rio.band(dst, 1),
            src_transform=src_transform,
            src_crs=crs,
            dst_transform=dst_transform,
            dst_crs=crs,
            resampling=rasterio.warp.Resampling.bilinear
        )